In [60]:
import os, tarfile, re
from typing import Union, List
import folia.main as folia
from folia import fql
import spacy
nlp = spacy.load("nl_core_news_sm")

### Extract from .tgz file

In [4]:
file = tarfile.open("20150602_SoNaRCorpus_NC_1.2.1.tgz")

In [18]:
select = [tarinfo for tarinfo in file if (tarinfo.name.startswith('./SoNaRCorpus_NC_1.2/SONAR500/FoLiA/WR-P-P-G_newspapers/00') and not tarinfo.name.startswith('./SoNaRCorpus_NC_1.2/SONAR500/FoLiA/WR-P-P-G_newspapers/000'))]

In [ ]:
file.extractall(path='output',members=select)

### Read .folia file  
https://foliapy.readthedocs.io/en/latest/folia.html#  
Get text: `txt = doc.text()`  
Print text: `print(doc)`

In [7]:
# base = './output/SoNaRCorpus_NC_1.2/SONAR500/FoLiA/WR-P-P-G_newspapers/'
base = '../NLNews/WR-P-P-G_newspapers/'

In [39]:
def create_segment(doc: Union[folia.Document, str]) -> str:
    """
    Returns the text as a single segment, filtering out paragraphs containing 10 or less words.
    """
    if isinstance(doc, str):
        doc = folia.Document(file=doc)
    paragraphs = [p.text() for p in doc.select(folia.Paragraph) if p.count(folia.Word) > 10]

    # Sometimes articles start with: "LOCATION -"; the following removes it.
    # Look for '-' in first sentence of the first paragraph.
    query = fql.Query(f'SELECT w WHERE text = "-" IN ID {doc.paragraphs(0)[0].id}')
    found = query(doc)
    if found:
        pp = doc.paragraphs(0).copy()
        word = found[0]
        # Walk backwards, only if the word directly before the '-' is a capitalized location.
        while True:
            prev = word.previous(folia.Word)
            if prev is None:
                pp[0].remove(word)
                break
            if prev.annotation(folia.PosAnnotation, set="http://ilk.uvt.nl/folia/sets/frog-mbpos-cgn").cls == 'SPEC(deeleigen)' and prev.text().isupper():
                pp[0].remove(word)
                word = prev
                prev = word.previous()
            else:
                break
        paragraphs[0] = pp.text()
    res = ' '.join(paragraphs)
    return res, {'id': doc.id, 'len': len(list(nlp(res).sents))}

In [ ]:
# Check if any file contains multiple headers (probably none)
# TAKES A LONG TIME
for root, dirs, files in os.walk(base+'000/', topdown=False):
   for file in files:
      if file.endswith('.xml'):
          d = folia.Document(file=os.path.join(root,file))
          hc = d.count(folia.Head)
          if hc > 1:
              print(f'{os.path.join(root,file)} - {hc}')

In [13]:
doc = folia.Document(file=base+'000/WR-P-P-G-0000000004.folia.xml')
xx, _ = create_segment(doc)
[x.count(folia.Sentence) for x in doc.select(folia.Paragraph)]

[5]

In [17]:
len(list(nlp(xx).sents))

4

### Create multi-segment documents

In [4]:
import numpy as np
from datetime import datetime
from tqdm import tqdm
rng = np.random.default_rng()

In [108]:
# List containing the location of all files
locs = [os.path.join(root,file) for root, _, files in os.walk(base) for file in files if file.endswith('.xml')]

In [54]:
# def create_doc(locs: Union[List[str],np.ndarray[str]]): # This type of hint only works in Python3.9 for whatever reason
def create_doc(locs, save=True):
    """
    Creates a folder containing .txt files from concatenated articles.
    :param locs: List/np.ndarray containing locations to the .xml files to be processed.
    :param save: Whether to save to a folder or to print out results.
    """
    if isinstance(locs, List):
        locs = np.array(locs)

    dt = datetime.now().strftime("%d%m-%H%M")
    i = 0
    pad = int(np.log10(len(locs))) + 1

    doc_lengths = []
    sect_lengths = []

    if save:
        new_dir = f'../NLNews/data_{dt}'
        os.mkdir(new_dir)
    else:
        docs = []

    with tqdm(total=locs.size, desc='Articles processed') as pbar:
        while locs.size > 0:
            if locs.size >= 6:
                n = rng.integers(2,5)
            elif locs.size == 5:
                n = rng.integers(2,4)
            else:
                n = locs.size
            slice = rng.choice(locs.size, size=n, replace=False)

            texts = [create_segment(t) for t in locs[slice]]
            locs = np.delete(locs, slice)
            doc = ''
            l = []
            for text, stats in texts:
                doc += f'==={stats["id"]}===\n{text}\n'
                l.append(stats["len"])
                sect_lengths.append(stats["len"])
            doc_lengths.append(sum(l))

            # doc = '==='+'\n==='.join(f'{stats["id"]}===\n{text}' for text, stats in texts)
            if save:
                with open(f'{new_dir}/{i:0{pad}}.txt', 'w') as tfile:
                    tfile.write(doc)
            else:
                docs.append(doc)
            i += 1
            pbar.update(n)

    print(f'Created {i} Documents')
    print(f'   Average document length = {np.mean(doc_lengths)}')
    print(f'   Average section length  = {np.mean(sect_lengths)}')

    if save is False: 
        return docs

In [97]:
create_doc([os.path.join(root,file) for root, _, files in os.walk(base+'000/') for file in files if file.endswith('.xml')])

Articles processed: 100%|██████████| 4096/4096 [27:36<00:00,  2.47it/s]

Created 1357 Documents
   Average document length = 40.171702284450994
   Average section length  = 13.308837890625


In [90]:
dd = create_doc(locs=["../NLNews/WR-P-P-G_newspapers/000/WR-P-P-G-0000000003.folia.xml","../NLNews/WR-P-P-G_newspapers/000/WR-P-P-G-0000000004.folia.xml"],save=False)

Articles processed: 100%|██████████| 2/2 [00:00<00:00,  3.09it/s]

Created 1 Documents
   Average document length = 25.0
   Average section length  = 12.5


In [68]:
def remove_info(text):
    """
    Removes lines starting with "===" from string.
    """
    return re.sub(r'^===.*\n', '', dd[0], flags=re.MULTILINE)

### Segmentation demo

In [73]:
import nltk
from nltk.tokenize import texttiling

In [85]:
re.sub('\n','\n\n',remove_info(dd[0]))

'Oud-minister van justitie mr. C.H.F. Polak , die voorzitter is van de Nederlandse Juristen Vereniging heeft een beroep gedaan op de politieke partijen ervoor te zorgen dat bij de komende verkiezingen de Tweede Kamer wordt versterkt met een aantal bekwame juristen . Hij vindt de teruggang van het aantal juristen in het parlement , juist nu de wetgever zich opmaakt tot grote veranderingen in het recht , zorgwekkend . " Misschien lijkt velen van u mijn beduchtheid voor het gehalte van onze rechtstaat wat overdreven . Maar de wetten worden onoverzichtelijk en tegenstrijdig . De vergunningenstelsels stapelen zich op , de wetshandhaving wordt grilliger en zwakker , overheidssteun aan noodlijdende bedrijven wordt verleend volgens vage , niet vastgelegde criteria . Enig beroep is niet mogelijk en zelfs het parlement mag niet weten welke bedrijven steun ontvangen " aldus mr. Polak , lid van de Eerste Kamer .\n\nDe zon verzuimt ook dit jaar niet . Hij schijnt op de akkers van Midden-Anatolië . 

In [94]:
tt = texttiling.TextTilingTokenizer(w=10,k=5,demo_mode=True, stopwords=nltk.corpus.stopwords.words('dutch'))
s, ss, d, b= tt.tokenize(re.sub('\n','\n\n',remove_info(dd[0])))
b

[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]

In [93]:
print(dd[0])

===WR-P-P-G-0000000003===
Na een slappe morgen kwam gisteren de effectenbeurs van Amsterdam pas in de late middag tot leven . Onder invloed van vooral Wall Street trokken de aandelenkoersen duidelijk aan . De belangrijkste graadmeter , de AEX-index , sloot 10,55 punten hoger op 573,50 . Dat kwam neer op een stijging van 1,87 procent . Op de laatste handelsdag van het kwartaal waren hoofdzakelijk de pensioenfondsen en verzekeraars actief . Zij wilden graag hun beleggingsfondsen nog even oppoetsen . De slotkoersen van de tweede driemaandsperiode gelden immers als peildatum . Dat leidde ertoe dat slechts twee van de hoofdfondsen met een koersverlies eindigden : Gucci en Hagemeyer . De brede opleving werd gedragen door de technologiefondsen . Winnaar onder de hoofdfondsen was KPNQwest dat 12,5 procent koerswinst noteerde . " Dit vind ik een van de meest ondergewaardeerde fondsen . Daar lopen de zaken voorspoedig " , meende een handelaar . Andere technologiefondsen die goede koerswinsten pa

In [96]:
ttt = texttiling.TextTilingTokenizer(w=10,k=5,demo_mode=False, stopwords=nltk.corpus.stopwords.words('dutch'))
len(ttt.tokenize(re.sub('\n','\n\n',remove_info(dd[0]))))

2